In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import random

# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [13]:
# 自定义 MAPE 损失函数
class MAPE_Loss(nn.Module):
    def __init__(self):
        super(MAPE_Loss, self).__init__()

    def forward(self, y_pred, y_true):
        epsilon = 1e-8  # 避免除零
        return torch.mean(torch.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# 自定义 RMSE 损失函数
class RMSE_Loss(nn.Module):
    def __init__(self):
        super(RMSE_Loss, self).__init__()

    def forward(self, y_pred, y_true):
        return torch.sqrt(torch.mean((y_true - y_pred) ** 2))

In [14]:
# 读取数据
data = pd.read_csv("../data/dataset.csv")

# 数据分割
data['target_class'] = pd.qcut(data['Cs'], q=10, labels=False)
X = data.drop(['Cs', 'target_class'], axis=1).values
y = data['Cs'].values
stratify_column = data['target_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=stratify_column)

# 数据标准化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 将数据转换为 PyTorch 张量并移动到 GPU
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

# 创建 DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [17]:
# 定义 ANN 模型（确保在使用前定义）
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 12)
        self.layer2 = nn.Linear(12, 90)
        self.layer3 = nn.Linear(90, 90)
        self.layer4 = nn.Linear(90, 60)
        self.layer5 = nn.Linear(60, 70)
        self.layer6 = nn.Linear(70, 30)
        self.output = nn.Linear(30, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = torch.relu(self.layer4(x))
        x = torch.relu(self.layer5(x))
        x = torch.relu(self.layer6(x))
        x = self.output(x)
        return x

# 初始化模型并将其移到 GPU（如果可用）
input_dim = X_train.shape[1]
model = ANN(input_dim).to(device)

# 定义损失函数和优化器
loss_function = MAPE_Loss().to(device)  # 自定义损失函数也要移到 GPU
optimizer = optim.Adam(model.parameters(), lr=0.001)

# # 设置 PyTorch 随机种子
# def set_random_seed(seed):
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)  # 如果使用 GPU
#     np.random.seed(seed)  # 设置 Numpy 随机种子
#     random.seed(seed)  # 设置 Python 原生随机数生成器的种子
#     torch.backends.cudnn.deterministic = True  # 保证卷积操作的确定性
#     torch.backends.cudnn.benchmark = False  # 禁用 cudnn 自动优化（用于固定输入尺寸）
#
# # 设置种子
# set_random_seed(42)

In [7]:
# 训练模型+包含早停
num_epochs = 3000
patience = 100  # 允许的最大连续未改进 epoch 数
best_loss = float('inf')  # 初始时验证损失设置为正无穷
epochs_without_improvement = 0  # 连续未改进的 epoch 数

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        # 移动 batch 数据到 GPU
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = loss_function(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 打印每10个 epoch 的损失
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

    # 计算验证损失
    model.eval()
    with torch.no_grad():
        # 在 GPU 上进行预测
        y_val_pred = model(X_test_tensor).to(device)
        # 验证损失计算时，确保 y_test_tensor 也在同一个设备上
        y_test_tensor = y_test_tensor.to(device)
        val_loss = loss_function(y_val_pred, y_test_tensor).item()  # 计算验证损失

    # 判断验证损失是否改善
    if val_loss < best_loss:
        best_loss = val_loss
        epochs_without_improvement = 0  # 重置计数器
        # 保存最佳模型
        torch.save(model.state_dict(), "ann_best_model.pth")
    else:
        epochs_without_improvement += 1

    # 如果验证损失在一定次数的 epoch 内没有改进，则停止训练
    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

# 保存模型
torch.save(model.state_dict(), "ann_best_model.pth")

Epoch [10/3000], Loss: 20.6088
Epoch [20/3000], Loss: 14.3592
Epoch [30/3000], Loss: 15.2728
Epoch [40/3000], Loss: 14.1516
Epoch [50/3000], Loss: 10.6182
Epoch [60/3000], Loss: 9.2494
Epoch [70/3000], Loss: 8.8155
Epoch [80/3000], Loss: 8.9821
Epoch [90/3000], Loss: 9.1149
Epoch [100/3000], Loss: 7.4997
Epoch [110/3000], Loss: 8.7275
Epoch [120/3000], Loss: 6.8173
Epoch [130/3000], Loss: 6.7137
Epoch [140/3000], Loss: 6.3701
Epoch [150/3000], Loss: 5.4745
Epoch [160/3000], Loss: 5.4778
Epoch [170/3000], Loss: 6.1108
Epoch [180/3000], Loss: 5.5141
Epoch [190/3000], Loss: 5.4048
Epoch [200/3000], Loss: 4.5437
Epoch [210/3000], Loss: 5.6263
Epoch [220/3000], Loss: 6.7235
Epoch [230/3000], Loss: 4.8358
Epoch [240/3000], Loss: 6.2325
Epoch [250/3000], Loss: 5.0213
Epoch [260/3000], Loss: 4.8752
Epoch [270/3000], Loss: 5.4204
Epoch [280/3000], Loss: 4.8591
Epoch [290/3000], Loss: 4.9229
Epoch [300/3000], Loss: 5.1672
Epoch [310/3000], Loss: 4.7501
Epoch [320/3000], Loss: 4.6002
Epoch [330/3

In [18]:
from function import metrics_to_dataframe

# 重新定义模型结构
model = ANN(input_dim).to(device)  # 使用相同的模型结构

# 加载模型参数，确保使用 weights_only=True 来提高安全性
model.load_state_dict(torch.load("ann_model.pth", weights_only=True))  # 加载模型参数

# 评估模型 (确保在 GPU 上)
model.eval()
with torch.no_grad():
    y_train_pred = model(X_train_tensor).cpu().numpy()  # 转换为 CPU 数据，便于后续处理
    y_test_pred = model(X_test_tensor).cpu().numpy()

# 计算并显示评估指标
ann_metrics = metrics_to_dataframe(y_train, y_train_pred, y_test, y_test_pred, 'ANN')
ann_metrics

,model,R2_train,MAE_train,MAPE_train,RMSE_train,R2_test,MAE_test,MAPE_test,RMSE_test
0,ANN,0.943932,5.739903,6.449726,9.452667,0.963017,5.347572,6.180497,7.632014


In [19]:
ann_train = pd.DataFrame({'Actual': y_train, 'Predicted': y_train_pred.squeeze()})
ann_test = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred.squeeze()})
ann_train.to_csv('ann_train.csv', index=False)
ann_test.to_csv('ann_test.csv', index=False)